In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
dataset = pd.read_csv('dataset.csv')
print(dataset.head)
print(dataset.columns)
filtered_dataset = dataset[(dataset[' Label'].isin(['DDoS', 'BENIGN']))]
# & (dataset[' Destination Port'] == 80)
filtered_dataset.to_csv('filtered_dataset.csv', index=False)
print("dataset is saved as csv file")

filtered_dataset = pd.read_csv('filtered_dataset.csv')




X = filtered_dataset.drop(columns=[' Label'])  # Assuming 'label' is the column containing labels
y = filtered_dataset[' Label']
# Convert label to numerical using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("Target feature is ",y_encoded)
# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with 0 or any other appropriate strategy
X.fillna(0, inplace=True)

# Perform feature scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print("X_scaled is ",X_scaled)
#Now we are going for feature selection

selector = SelectKBest(chi2, k=10)
X_selected = selector.fit_transform(X_scaled, y_encoded)
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_feature_indices]
print("Selected Features:")
for feature_name in selected_feature_names:
    print(feature_name)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)
isolation_forest = IsolationForest(contamination=0.1)
print("Splitted the dataset for testig and training")
print(X_train.shape)
print(X_test.shape)
# Now we need to implement isolation forest

isolation_forest = IsolationForest(contamination=0.1)
isolation_forest.fit(X_selected)
print("Training completed")
anomaly_preds = isolation_forest.predict(X_selected)
anomaly_preds_binary = [1 if x == -1 else 0 for x in anomaly_preds]
print(anomaly_preds_binary)
print("Now starting to test the model")

anomaly_preds = isolation_forest.predict(X_test)
anomaly_preds_binary = [1 if x == -1 else 0 for x in anomaly_preds]

# Calculate metrics
f1 = f1_score(y_test, anomaly_preds_binary, average='weighted')
precision = precision_score(y_test, anomaly_preds_binary, average='weighted')
recall = recall_score(y_test, anomaly_preds_binary, average='weighted')


# Print the results
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


<bound method NDFrame.head of          Destination Port   Flow Duration   Total Fwd Packets  \
0                   54865               3                   2   
1                   55054             109                   1   
2                   55055              52                   1   
3                   46236              34                   1   
4                   54863               3                   2   
...                   ...             ...                 ...   
242844                443         5325752                   7   
242845                 53           24005                   2   
242846                 53             202                   2   
242847                 53        23648117                   2   
242848                 53        45112564                   2   

         Total Backward Packets  Total Length of Fwd Packets  \
0                             0                           12   
1                             1                            6 